In [3]:
import pandas as pd 
import numpy as np
from sklearn.preprocessing import MinMaxScaler

In [4]:
### Load label
labels = pd.read_csv("../data/label.csv")

### load data
data = pd.read_csv("../data/sensor_log.csv")

In [3]:
labels.head()

,block_id,anomalous
0,144,0
1,145,0
2,146,0
3,147,0
4,148,0


In [4]:
data.head()

,timestamp,block_id,sensor_00,sensor_01,sensor_02,sensor_03,sensor_04,sensor_05,sensor_06,sensor_07,sensor_08,sensor_09
0,2018-04-01 00:00:00,0,2.465394,47.09201,53.2118,46.310760,634.3750,76.45975,13.41146,16.13136,15.56713,15.05353
1,2018-04-01 00:01:00,0,2.465394,47.09201,53.2118,46.310760,634.3750,76.45975,13.41146,16.13136,15.56713,15.05353
2,2018-04-01 00:02:00,0,2.444734,47.35243,53.2118,46.397570,638.8889,73.54598,13.32465,16.03733,15.61777,15.01013
3,2018-04-01 00:03:00,0,2.460474,47.09201,53.1684,46.397568,628.1250,76.98898,13.31742,16.24711,15.69734,15.08247
4,2018-04-01 00:04:00,0,2.445718,47.13541,53.2118,46.397568,636.4583,76.58897,13.35359,16.21094,15.69734,15.08247


In [5]:
data.describe()

,block_id,sensor_00,sensor_01,sensor_02,sensor_03,sensor_04,sensor_05,sensor_06,sensor_07,sensor_08,sensor_09
count,192830.000000,192830.000000,192830.000000,192830.000000,192830.000000,192830.000000,192830.000000,192830.000000,192830.000000,192830.000000,192830.000000
mean,9641.000000,2.459135,48.153622,51.790473,44.226571,633.010905,75.725928,13.943126,16.195575,15.489414,15.101593
std,5566.537047,0.057049,2.144784,1.591106,1.549972,11.372609,7.241880,0.662427,0.370934,0.361631,0.225652
min,0.000000,0.000000,37.065970,42.317710,34.982640,3.104745,0.000000,0.014468,0.028935,0.028935,0.007234
25%,4820.000000,2.448669,46.701390,50.781250,43.185760,628.935200,70.811155,13.411460,15.957750,15.227140,15.053530
50%,9641.000000,2.456539,48.263885,51.866320,44.357635,633.680500,75.749880,13.686340,16.167530,15.494790,15.082470
75%,14462.000000,2.500810,49.522570,52.951385,45.355900,638.194500,80.478037,14.576100,16.427950,15.697340,15.118630
max,19282.000000,2.585285,60.237007,56.032990,50.204142,690.462452,113.435420,15.964990,17.502495,18.695345,16.399020


In [6]:
blockid_train = labels['block_id'].values

In [7]:
_train = data['block_id'].isin(blockid_train)
data_train = data[_train]
data_test = data[~_train]

In [8]:
anomalous = labels['anomalous'].values

In [9]:
unique, count = np.unique(anomalous, return_counts=True)
print(unique)
print(count)

[0 1]
[5617  164]


In [10]:
data_train.to_csv('../data/train.csv', index=False)
data_test.to_csv('../data/test.csv', index=False)

### Scale Min-Max

In [11]:
scaler = MinMaxScaler()

In [12]:
columns = list(data.columns)

In [13]:
train_values = data_train.values[:, 2:]
test_values = data_test.values[:, 2:]

block_train_values = np.expand_dims(data_train.values[:, 1], axis=1)
block_test_values = np.expand_dims(data_test.values[:, 1], axis=1)

timestemp_train_values = np.expand_dims(data_train.values[:, 0], axis=1)
timestemp_test_values = np.expand_dims(data_test.values[:, 0], axis=1)

In [14]:
train_values_scaled = scaler.fit_transform(train_values)
test_values_scaled = scaler.fit_transform(test_values)

In [15]:
timestemp_train_values.shape, block_train_values.shape, train_values_scaled.shape

((57810, 1), (57810, 1), (57810, 10))

In [16]:
train_values_all = np.hstack((timestemp_train_values, block_train_values, train_values_scaled))
test_values_all = np.hstack((timestemp_test_values, block_test_values, test_values))

In [17]:
df_train = pd.DataFrame(data=train_values_all, columns=columns)
df_test = pd.DataFrame(data=test_values_all, columns=columns)

In [18]:
df_train.to_csv('../data/train_normalize.csv', index=False)
df_test.to_csv('../data/test_normalize.csv', index=False)

In [19]:
len(df_train), len(df_test)

(57810, 135020)

In [22]:
df = pd.read_csv('../data/train_normalize.csv')
df.head()

,timestamp,block_id,sensor_00,sensor_01,sensor_02,sensor_03,sensor_04,sensor_05,sensor_06,sensor_07,sensor_08,sensor_09
0,2018-04-02 00:00:00,144,0.951342,0.341364,0.755172,0.741935,0.896145,0.791095,0.778336,0.581489,0.263138,0.488599
1,2018-04-02 00:01:00,144,0.951342,0.341364,0.755172,0.741935,0.896145,0.791095,0.778336,0.581489,0.263138,0.488599
2,2018-04-02 00:02:00,144,0.947537,0.337395,0.751724,0.741935,0.886458,0.796020,0.780570,0.570086,0.279793,0.488599
3,2018-04-02 00:03:00,144,0.953245,0.341364,0.751724,0.741935,0.881025,0.806212,0.775544,0.570086,0.263138,0.472316
4,2018-04-02 00:04:00,144,0.945634,0.331441,0.755172,0.741935,0.893546,0.782298,0.782245,0.570086,0.336418,0.488599


In [23]:
labels.head()

,block_id,anomalous
0,144,0
1,145,0
2,146,0
3,147,0
4,148,0


In [25]:
train_tmp = df.merge(labels,how='left',on='block_id')

In [5]:
labels.isnull().sum()

block_id     0
anomalous    0
dtype: int64

In [6]:
val = pd.read_csv('../data/val/val.csv')

In [7]:
val.head()

,Unnamed: 0,block_id,sensor_00_mean,sensor_00_std,sensor_00_min,sensor_00_max,sensor_00_sum,sensor_00_last,sensor_01_mean,sensor_01_std,...,target5,target6,target7,target8,target9,target10,target11,target12,target13,anomalous
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.410344,0.410344,0.410344,0.410344,0.410344,0.410344,0.410344,0.410344,0.410344,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.410344,0.410344,0.410344,0.410344,0.410344,0.410344,0.410344,0.410344,0.410344,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.410344,0.410344,0.410344,0.410344,0.410344,0.410344,0.410344,0.410344,0.410344,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.410344,0.410344,0.410344,0.410344,0.410344,0.410344,0.410344,0.410344,0.410344,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.410344,0.410344,0.410344,0.410344,0.410344,0.410344,0.410344,0.410344,0.410344,NaN


In [8]:
all_feature = pd.read_csv('../data/all_feature.csv')

In [11]:
all_feature.shape

(19283, 361)